# Test BigData

In [1]:
import os
import time
# import azure.cosmos
# import azure.cosmos.cosmos_client as cosmos_client
from runpy import run_path
from azure.cosmos import exceptions, CosmosClient, PartitionKey # https://github.com/Azure-Samples/azure-cosmos-db-python-getting-started/blob/master/cosmos_get_started.py

credentials = run_path("/home/cosmosuser/notebooks/credentials.py")

endpoint = credentials['HOST']
key = credentials['MASTER_KEY']



In [2]:
# <create_cosmos_client>
client = CosmosClient(endpoint, key)
# </create_cosmos_client>

# Create a database
# <create_database_if_not_exists>
database_name = 'mybigdata'
database = client.create_database_if_not_exists(id=database_name)
# </create_database_if_not_exists>

## Read and process data line by line

In [ ]:

# jupyter nbconvert --to script BigData.ipynb
# Read a CSV file in Python, libe-by-line, by Jeff Heaton (http://www.jeffheaton.com/tutorials/)
import codecs
import csv

FILENAME = "/home/cosmosuser/notebooks/census/Rate.csv" # https://www.kaggle.com/hhs/health-insurance-marketplace#Rate.csv
ENCODING = 'utf-8'

with codecs.open(FILENAME, "r", ENCODING) as fp:
    reader = csv.reader(fp)

    # read CSV headers
    headers = next(reader)
    print(headers)

    # read rest of file
    for row in reader:
        # Print each row
        print(row)

        # Print individual fields of the row
        # print("{},{},{},{} = {}".format(row[0],row[1],row[2],row[3],row[4]))